<img src=../figures/Brown_logo.svg width=50%>

## Data-Driven Design & Analyses of Structures & Materials (3dasm)

## Lecture 20


### Miguel A. Bessa | <a href = "mailto: miguel_bessa@brown.edu">miguel_bessa@brown.edu</a>  | Associate Professor

## Outline of today

Provide an overview of your Final Project for the course.

This final project is inspired by the following research article:

* **Bessa, M. A., Glowacki, P., & Houlder, M. (2019). Bayesian Machine Learning in Metamaterial Design: Fragile Becomes Supercompressible. Advanced Materials, 31(48), 1–6. https://doi.org/10.1002/adma.201904845**

It focused on transforming a fragile polymer material used in 3D printing into a supercompressible metamaterial by designing its geometry.

The data was generated by Finite Element Analyses (using [ABAQUS](https://www.3ds.com/products/simulia/abaqus) together with `f3dasm`).

However, you will not need to generate the data using ABAQUS. Instead, you will be provided with a dataset. This way, you can focus on the Machine Learning and Optimization tasks and avoid running time-consuming simulations.

## Designing supercompressible metamaterials

<img src=../figures/supercompressible_process.png width=65%, align='left'>

<img src=../figures/supercompressible_schematic.png width=30%, align='right'>

<!--- <br><br><br><br><br><br><br><br><br><br><br><br><br><br><br> --->

* Parametrizing your structure from a concept
* Sampling designs
* Run finite element simulation (ABAQUS)
* Learn the objective for the full range of designs
* Manufacture your found optimum

## Datasets

The supercompressible meta-material is parameterized by **5 geometric parameters and 2 material parameters**:
* The geometry is defined by the top and bottom diameters, $D_1$ and $D_2$, 
* the height $P$
* the cross-section parameters of the vertical longerons: area $A$, 
* moments of inertial $I_x$ and $I_y$, 
* torsional constant $J$,. 
* The isotropic material is defined by its elastic constants: Young's modulus $E$ and shear modulus $G$.

$\frac{D_1-D_2}{D_1},\ \frac{P}{D_1},\ \frac{I_x}{D_1^4},\ \frac{I_y}{D_1^4},\ \frac{J}{D_1^4},\ \frac{A}{D_1^2}, \frac{G}{E}$

A dataset with 50000 experiments that is parametrized by 7 parameters (`supercompressible_7d`) will be provided.

This is a **7-dimensional problem** and learning the response surface may require a **significant amount of training points**

Therefore, you will also consider a **simpler version** of the problem in 3 dimensions, defined by constraining the longerons' cross-section to be circular with diameter $d$, and choosing a particular material, leading to the following 3 features:

$\frac{d}{D_1}, \frac{D_2-D_1}{D_1},\ \frac{P}{D_1}$

This is the `supercompressible_3d` dataset with 1000 experiments

## Output data

* We can use **non-linear finite element analyses** to predict the **complete buckling and post-buckling behavior**. 
* From the analyses, we can understand if a **material is coilable**, compute the **critical buckling stress** $\sigma_{crit}$ and the **energy absorbed** $E_{abs}$.
* Due to unsuccessful simulations, there are **missing points in the datasets** in the data).

## How do we normally create this dataset?

<img src="../figures/blocks_supercompressible_original.png" width="50%">

Some problems may arise:
* How do we **parametrize** the design efficiently?
* How do we run this in **parallel** or on a **high computation cluster**?
* How do we **avoid reinventing the wheel** (e.g. calling Abaqus with python scripts)

## How is the dataset created with f3dasm?

<img src="../figures/supercompressible_colored.png" width="100%">

First we import some packages ..

In [7]:
from f3dasm.design import Domain
from f3dasm import ExperimentData
import numpy as np
from pathlib import Path

We define the number of experiments, the pseudo-random number generator seed and if the design is circular or not

In [3]:
SEED = 55
N_EXPERIMENTS = 2
CIRCULAR = True

np.random.seed(SEED)

We create the empty domain object:

In [ ]:
domain = f3dasm.Domain()

.. and we add the parameters to our domain:

In [4]:
domain.add_constant('young_modulus', value=3500.0)
domain.add_constant('n_longerons', value=3)
domain.add_constant('bottom_diameter', value=100.0)

domain.add_float('ratio_top_diameter', low=0.0, high=0.8)
domain.add_float('ratio_pitch', low=0.25, high=1.5)

if CIRCULAR:
    domain.add_float('ratio_d', low=0.004, high=0.073)
    domain.add_constant('ratio_shear_modulus', value=0.3677) 

In [8]:
domain

Domain(space={'young_modulus': ConstantParameter(value=3500.0), 'n_longerons': ConstantParameter(value=3), 'bottom_diameter': ConstantParameter(value=100.0), 'ratio_top_diameter': ContinuousParameter(lower_bound=0.0, upper_bound=0.8, log=False), 'ratio_pitch': ContinuousParameter(lower_bound=0.25, upper_bound=1.5, log=False), 'ratio_d': ContinuousParameter(lower_bound=0.004, upper_bound=0.073, log=False), 'ratio_shear_modulus': ConstantParameter(value=0.3677)})

We sample from the domain using the [Sobol Sequence sampler](https://salib.readthedocs.io/en/latest/_modules/SALib/sample/sobol_sequence.html):

In [10]:
experimentdata = ExperimentData(domain)
experimentdata.sample(sampler='sobol', n_samples=N_EXPERIMENTS, seed=SEED)

In [11]:
experimentdata

After that, we add another parameter to our domain (`imperfection`) which will be sampled from a lognormal distribution:

In [13]:
experimentdata.add_input_parameter('imperfection', f3dasm.ContinuousParameter())

In [15]:
sampled_imperfections = np.random.lognormal(-2.705021452041446, 0.293560379208524, N_EXPERIMENTS)
experimentdata.input_data.data['imperfection'] = sampled_imperfections

In [16]:
experimentdata

Now we are ready to evaluate our samples with a datagenerator:
The simulation consists of two concurrent ABAQUS simulations:

* The first one is a linear buckling simulation
* The second one is a RIKS simulation

In [18]:
from f3dasm.datageneration.abaqus import pre_process, post_process, AbaqusSimulator

We split up the components of the simulation in three parts:

* **pre-processing**: combine the design and the other parameters/constants to an input file
* **processing**: run the simulation
* **post-processing**: convert the output of the simulator back to `f3dasm`

<img src="../figures/blocks_supercompressible_datagenerator.png" width="100%">

In [19]:
# Buckling
buckling_script_python_file = 'supercompressible_lin_buckle'
buckling_function_name_execute = 'main'
buckling_script_parent_folder_path = Path(r'H:\GitHub\f3dasm_simulate\notebooks')
buckling_post_python_file = 'supercompressible_lin_buckle_pp'
buckling_function_name_post = 'main'

simulator_buckling = AbaqusSimulator(name = "Simul_SUPERCOMPRESSIBLE_LIN_BUCKLE", 
                                     delete_odb=False, num_cpus=1)

simulator_buckling.add_pre_process(pre_process, folder_path=buckling_script_parent_folder_path,   
                                   python_file=buckling_script_python_file, 
                                   function_name=buckling_function_name_execute,
                                   circular=CIRCULAR,
                                   name = "Simul_SUPERCOMPRESSIBLE_LIN_BUCKLE",
                                   remove_temp_files=False)

simulator_buckling.add_post_process(post_process, folder_path=buckling_script_parent_folder_path, 
                                    python_file=buckling_post_python_file, 
                                    function_name=buckling_function_name_post,
                                    name = "Simul_SUPERCOMPRESSIBLE_LIN_BUCKLE",
                                    remove_temp_files=False)

In [20]:
# RIKS
riks_script_python_file = 'supercompressible_riks'
riks_function_name_execute = 'main'
riks_script_parent_folder_path = Path(r'H:\GitHub\f3dasm_simulate\notebooks')
riks_post_python_file = 'supercompressible_riks_pp'
riks_function_name_post = 'main'

simulator_riks = AbaqusSimulator(name = "Simul_SUPERCOMPRESSIBLE_RIKS", 
                                     delete_odb=False, num_cpus=1, delete_temp_files=False)

simulator_riks.add_pre_process(pre_process, folder_path=riks_script_parent_folder_path, 
                               python_file=riks_script_python_file, 
                               function_name=riks_function_name_execute,
                               circular=CIRCULAR,
                               name = "Simul_SUPERCOMPRESSIBLE_RIKS",
                               remove_temp_files=False)

simulator_riks.add_post_process(post_process, folder_path=riks_script_parent_folder_path, 
                                python_file=riks_post_python_file, 
                                function_name=riks_function_name_post,
                                name = "Simul_SUPERCOMPRESSIBLE_RIKS",
                                remove_temp_files=False)

Now we can run the simulators:
First the buckling simulation:

```
data.evaluate(simulator_buckling, mode='sequential')
```

From this simulation we will know if the design we are testing is coilable or not.
Only designs that are coilable will be subjected to the RIKS simulation

```
riks_selection = [index for index, 
                    sample in enumerate(data.output_data) 
                    if sample['coilable']]
```

We mark the selected jobs 'open' again so that the `evaluate` method will evalute them!

```
experimentdata.mark(riks_selection, 'open')

experimentdata.evaluate(simulator_riks, mode='cluster')
```

This is how the data for the 7 parameters looks like:

In [25]:
ExperimentData.from_file('/home/martin/Documents/GitHub/3dasm_course/Projects/Supercompressible/STUFF_NOT_TO_UPLOAD/data/supercompressible_7d')

## Let's do machine learning

Now that we have evaluated samples from our design space, we are trying to predict the responses for the entire domain with a machine learning model:

* The **coilability** with a **classification model**
* The critical buckling load and energy with a **regression model**


<img src=../figures/regression_classification.png width=60%>

## Aim of the project

* Find good **regression** and **classification** models to predict the coilability, $\sigma_{crit}$ and $E$ 
* Investigate the influence of the number of training points and hyperparameters
* Optimize your models to find improved designs!